In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import xarray as xr
import ghibtools as gh
import tqdm
from params import *

%matplotlib inline

## PARAMS

In [3]:
eeg_chans = ['Fp2-C4','C4-T4','T4-O2','Fz-Cz','Cz-Pz','Fp1-C3','C3-T3','T3-O1']
srate = 256

In [4]:
run_all = True

In [5]:
tf_mode = 'moi'
# tf_mode = 'cmo"

In [7]:
f_start = 10
f_stop = 20
n_steps = 40

cycle_start = f_start
cycle_stop = f_stop

TOOLS

GO

In [20]:
def data_to_tf(patient, srate=srate, f_start = f_start , f_stop=f_stop, n_steps=n_steps, save=True):
    data_file = f'../dataarray/da_staged_{patient}.nc'
    da = xr.load_dataarray(data_file)
    da = da.sel(stage = stage_to_study).dropna(dim='time')
    chans = list(da.coords['chan'].values)
    
    concat = []
        
    for chan in eeg_chans:
        sig = da.loc[chan, :].dropna(dim='time').values
        tf_matrix =  gh.tf(sig=sig, srate=srate, f_start=f_start , f_stop=f_stop, n_steps=n_steps, cycle_start=cycle_start, cycle_stop=cycle_stop)    
        concat.append(tf_matrix)
        
    da_tf = xr.concat(concat, dim = 'chan')
    da_tf = da_tf.assign_coords({'chan':eeg_chans})
    
    if save:
        da_tf.to_netcdf(f'../dataarray/da_tf_{patient}.nc')
    return da_tf

In [21]:
def tf_to_stretch_cycled(da_tf, patient, save=True):
    resp_features = pd.read_excel(f'../df_analyse/resp_features_{patient}.xlsx')

    concat = []
    for chan in eeg_chans:
        da_stretch_cycle = gh.tf_cycle_stretch(da=da_tf, chan=chan, rsp_features=resp_features, nb_point_by_cycle=1000, inspi_ratio = 0.4, save_path = f'../df_analyse/resp_features_new_{patient}.xlsx')
        concat.append(da_stretch_cycle)
        
    da_tf_stretched = xr.concat(concat, dim = 'chan')
    da_tf_stretched = da_tf_stretched.assign_coords({'chan':eeg_chans})
    if save:
        da_tf_stretched.to_netcdf(f'../dataarray/da_tf_stretched_{patient}.nc')
    return da_tf_stretched

In [22]:
if run_all:
    for patient in patients:
        print(patient)
        da_tf = data_to_tf(patient)
        da_tf_stretched = tf_to_stretch_cycled(da_tf , patient)
        del da_tf
        del da_tf_stretched

P1
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
P2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
P3
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
P4
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
n cycle removed = 1
P5
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
P6
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
n cycle removed = 2
P7
n cycle removed = 2

In [8]:
resp_features = pd.read_excel(f'../df_analyse/resp_features_{patient}.xlsx')

In [9]:
resp_features

,Unnamed: 0,patient,cycle_num,inspi_index,expi_index,inspi_time,expi_time,cycle_duration,insp_duration,exp_duration,cycle_freq,insp_volume,exp_volume,total_amplitude,insp_amplitude,exp_amplitude
0,0,P1,0,106,386,0.414062,1.507812,3.480469,1.093750,2.386719,0.287318,4.385588,-8.666698,16.760317,7.007745,9.752572
1,1,P1,1,997,1260,3.894531,4.921875,2.941406,1.027344,1.914062,0.339973,4.358856,-7.304945,14.472632,6.356609,8.116023
2,2,P1,2,1750,2036,6.835938,7.953125,2.921875,1.117188,1.804688,0.342246,5.878432,-8.636705,17.935857,7.655543,10.280315
3,3,P1,3,2498,2784,9.757812,10.875000,2.847656,1.117188,1.730469,0.351166,6.638856,-9.320204,24.845939,9.870429,14.975511
4,4,P1,4,3227,3497,12.605469,13.660156,2.710938,1.054688,1.656250,0.368876,9.314802,-10.707655,30.863582,15.854817,15.008765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,2347,P1,2347,1739402,1739660,6794.539062,6795.546875,2.714844,1.007812,1.707031,0.368345,3.146200,-3.676323,8.940561,4.945372,3.995189
2346,2348,P1,2348,1740097,1740345,6797.253906,6798.222656,2.652344,0.968750,1.683594,0.377025,3.486908,-4.390518,11.982176,5.627687,6.354489
2347,2349,P1,2349,1740776,1741032,6799.906250,6800.906250,2.507812,1.000000,1.507812,0.398754,2.909783,-3.793214,10.642699,4.902590,5.740108
2348,2350,P1,2350,1741418,1741675,6802.414062,6803.417969,2.445312,1.003906,1.441406,0.408946,2.453249,-3.601574,9.652076,4.328528,5.323548


In [ ]:
resp_features

In [10]:
for i , row in resp_features.iterrows():
    if i < 10:
        print(row)

Unnamed: 0                 0
patient                   P1
cycle_num                  0
inspi_index              106
expi_index               386
inspi_time          0.414062
expi_time           1.507812
cycle_duration      3.480469
insp_duration        1.09375
exp_duration        2.386719
cycle_freq          0.287318
insp_volume         4.385588
exp_volume         -8.666698
total_amplitude    16.760317
insp_amplitude      7.007745
exp_amplitude       9.752572
Name: 0, dtype: object
Unnamed: 0                 1
patient                   P1
cycle_num                  1
inspi_index              997
expi_index              1260
inspi_time          3.894531
expi_time           4.921875
cycle_duration      2.941406
insp_duration       1.027344
exp_duration        1.914062
cycle_freq          0.339973
insp_volume         4.358856
exp_volume         -7.304945
total_amplitude    14.472632
insp_amplitude      6.356609
exp_amplitude       8.116023
Name: 1, dtype: object
Unnamed: 0                